In [1]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf

In [2]:
# ✅ Enhanced Data Generator (Fix bugs & normalize labels)

class AgeGenderRaceDataGenerator(Sequence):
    def __init__(self, file_list, folder_path, batch_size=32, image_size=(96, 96), shuffle=True):
        self.file_list = file_list
        self.folder_path = folder_path
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.file_list) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.file_list)

    def __getitem__(self, index):
        batch_files = self.file_list[index * self.batch_size:(index + 1) * self.batch_size]

        images, ages, genders, races = [], [], [], []

        for fname in batch_files:
            img_path = os.path.join(self.folder_path, fname)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img = cv2.resize(img, self.image_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.astype(np.float32) / 255.0

            try:
                name_parts = fname.split('_')
                age = float(name_parts[0])
                gender = int(name_parts[1])
                race = int(name_parts[2])
                if gender not in [0, 1] or race not in [0, 1, 2, 3, 4]:
                    continue
            except:
                continue

            images.append(img)
            ages.append(age)
            genders.append(gender)
            races.append(race)

        return np.array(images), {
            'age_output': np.array(ages, dtype=np.float32),
            'gender_output': np.array(genders, dtype=np.int32),
            'race_output': np.array(races, dtype=np.int32)
        }


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import (Input, Dense, GlobalAveragePooling2D, 
                                     Dropout, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np

# Optional: Focal loss for gender
def focal_loss(gamma=2., alpha=.25):
    def loss_fn(y_true, y_pred):
        bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)
        bce_exp = tf.exp(-bce)
        return alpha * (1 - bce_exp) ** gamma * bce
    return loss_fn

# Create the model
def create_model(input_shape=(96, 96, 3), num_races=5):
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomContrast(0.1)
    ])

    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs)

    base = MobileNetV2(include_top=False, input_shape=input_shape, weights='imagenet')
    base.trainable = True
    x = base(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)

    # Age Head
    x_age = Dense(128, activation='relu')(x)
    x_age = Dropout(0.3)(x_age)
    x_age = Dense(64, activation='relu')(x_age)
    age_output = Dense(1, name='age_output')(x_age)

    # Gender Head
    x_gender = Dense(128, activation='relu')(x)
    x_gender = BatchNormalization()(x_gender)
    x_gender = Dropout(0.4)(x_gender)
    x_gender = Dense(64, activation='relu')(x_gender)
    gender_output = Dense(1, activation='sigmoid', name='gender_output')(x_gender)

    # Race Head
    x_race = Dense(128, activation='relu')(x)
    x_race = BatchNormalization()(x_race)
    x_race = Dropout(0.4)(x_race)
    race_output = Dense(num_races, activation='softmax', name='race_output')(x_race)

    model = Model(inputs=inputs, outputs=[age_output, gender_output, race_output])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss={
            'age_output': Huber(),
            'gender_output': focal_loss(),
            'race_output': 'sparse_categorical_crossentropy'
        },
        loss_weights={
            'age_output': 2.0,
            'gender_output': 2.0,
            'race_output': 1.0
        },
        metrics={
            'age_output': ['mae'],
            'gender_output': ['accuracy'],
            'race_output': ['accuracy']
        }
    )
    return model

In [4]:
# ✅ Usage
folder =  r'C:\ageimg'
file_list = [f for f in os.listdir(folder) if f.endswith('.jpg') or f.endswith('.png')]

In [5]:
np.random.shuffle(file_list)
split = int(len(file_list) * 0.8)
train_files = file_list[:split]
val_files = file_list[split:]

In [6]:
train_gen = AgeGenderRaceDataGenerator(train_files, folder)
val_gen = AgeGenderRaceDataGenerator(val_files, folder, shuffle=False)

In [7]:
# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5, verbose=1)
]

# Usage (ensure you have train_gen and val_gen ready):
model = create_model()

# Optional: class weights if gender imbalance exists
gender_class_weight = {0: 1.0, 1: 2.0}  # Example
class_weights = {'gender_output': gender_class_weight}


In [9]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20
603/603 ━━━━━━━━━━━━━━━━━━━━ 863s 1s/step - age_output_loss: 25.6599 - age_output_mae: 26.1533 - gender_output_accuracy: 0.5863 - gender_output_loss: 0.0688 - loss: 53.5202 - race_output_accuracy: 0.3057 - race_output_loss: 2.0622 - val_age_output_loss: 10.0941 - val_age_output_mae: 10.5801 - val_gender_output_accuracy: 0.6613 - val_gender_output_loss: 0.0629 - val_loss: 21.7013 - val_race_output_accuracy: 0.4941 - val_race_output_loss: 1.3856 - learning_rate: 5.0000e-05
Epoch 2/20
603/603 ━━━━━━━━━━━━━━━━━━━━ 645s 1s/step - age_output_loss: 9.5268 - age_output_mae: 10.0090 - gender_output_accuracy: 0.6709 - gender_output_loss: 0.0475 - loss: 20.6428 - race_output_accuracy: 0.4675 - race_output_loss: 1.4941 - val_age_output_loss: 7.6623 - val_age_output_mae: 8.1388 - val_gender_output_accuracy: 0.7021 - val_gender_output_loss: 0.0424 - val_loss: 16.6389 - val_race_output_accuracy: 0.5690 - val_race_output_loss: 1.2297 - learning_rate: 5.0000e-05
Epoch 3/20
603/603 ━━━━━━━━━━

In [10]:
train_metrics = model.evaluate(train_gen, verbose=0)
val_metrics = model.evaluate(val_gen, verbose=0)

In [14]:
# Extract metrics from evaluation
metric_names = model.metrics_names
train_results = model.evaluate(train_gen, return_dict=True, verbose=0)
val_results = model.evaluate(val_gen, return_dict=True, verbose=0)

In [15]:
print("\n📊 Final Metrics:")
print(f"Train - Gender Acc: {train_results['gender_output_accuracy']:.4f} | Race Acc: {train_results['race_output_accuracy']:.4f} | Age MAE: {train_results['age_output_mae']:.2f}")
print(f"Valid - Gender Acc: {val_results['gender_output_accuracy']:.4f} | Race Acc: {val_results['race_output_accuracy']:.4f} | Age MAE: {val_results['age_output_mae']:.2f}")



📊 Final Metrics:
Train - Gender Acc: 0.8115 | Race Acc: 0.6519 | Age MAE: 6.01
Valid - Gender Acc: 0.8002 | Race Acc: 0.6256 | Age MAE: 7.41


In [24]:
import cv2
import numpy as np

# Modify these according to your dataset
race_labels = ['White', 'Black', 'Asian', 'Indian', 'Other']
gender_labels = ['Male', 'Female']

def predict_from_image(img_path, model, max_age=100):
    """
    Predict age, gender, and race from an image using the provided model.
    
    Args:
        img_path (str): Path to the image file.
        model (tf.keras.Model): Trained Keras model.
        max_age (int): Max age used during normalization (default 100).
    
    Returns:
        dict: Predictions including age, gender, and race.
    """

    # Load image
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"❌ Error: Could not load image from path: {img_path}")

    # Preprocess
    img = cv2.resize(img, (96, 96))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)

    # Predict
    age_pred, gender_pred, race_pred = model.predict(img, verbose=0)

    # Decode
    age = round(float(age_pred[0]) , 1)  # Denormalize if needed
    gender = gender_labels[int(gender_pred[0] > 0.5)]
    race = race_labels[np.argmax(race_pred[0])]

    # Print
    print("🎯 Prediction Results")
    print(f"🧒 Age: {age}")
    print(f"🚻 Gender: {gender}")
    print(f"🌍 Race: {race}")

    return {"age": age, "gender": gender, "race": race}
preds = predict_from_image("istockphoto-154946755-612x612.jpg", model, max_age=100)

🎯 Prediction Results
🧒 Age: 76.7
🚻 Gender: Female
🌍 Race: White


C:\Users\rites\AppData\Local\Temp\ipykernel_12028\3056239620.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = round(float(age_pred[0]) , 1)  # Denormalize if needed
C:\Users\rites\AppData\Local\Temp\ipykernel_12028\3056239620.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gender = gender_labels[int(gender_pred[0] > 0.5)]


In [25]:
model.save("age_gender_race_model.h5")